In [67]:
import sys
from random import sample
import math

In [2]:
fp = open('MQ2008/Fold1/train.txt', 'r')
data = fp.readlines()
print(data[0])

0 qid:10002 1:0.007477 2:0.000000 3:1.000000 4:0.000000 5:0.007470 6:0.000000 7:0.000000 8:0.000000 9:0.000000 10:0.000000 11:0.471076 12:0.000000 13:1.000000 14:0.000000 15:0.477541 16:0.005120 17:0.000000 18:0.571429 19:0.000000 20:0.004806 21:0.768561 22:0.727734 23:0.716277 24:0.582061 25:0.000000 26:0.000000 27:0.000000 28:0.000000 29:0.780495 30:0.962382 31:0.999274 32:0.961524 33:0.000000 34:0.000000 35:0.000000 36:0.000000 37:0.797056 38:0.697327 39:0.721953 40:0.582568 41:0.000000 42:0.000000 43:0.000000 44:0.000000 45:0.000000 46:0.007042 #docid = GX008-86-4444840 inc = 1 prob = 0.086622



In [15]:
print(data[0].split())

['0', 'qid:10002', '1:0.007477', '2:0.000000', '3:1.000000', '4:0.000000', '5:0.007470', '6:0.000000', '7:0.000000', '8:0.000000', '9:0.000000', '10:0.000000', '11:0.471076', '12:0.000000', '13:1.000000', '14:0.000000', '15:0.477541', '16:0.005120', '17:0.000000', '18:0.571429', '19:0.000000', '20:0.004806', '21:0.768561', '22:0.727734', '23:0.716277', '24:0.582061', '25:0.000000', '26:0.000000', '27:0.000000', '28:0.000000', '29:0.780495', '30:0.962382', '31:0.999274', '32:0.961524', '33:0.000000', '34:0.000000', '35:0.000000', '36:0.000000', '37:0.797056', '38:0.697327', '39:0.721953', '40:0.582568', '41:0.000000', '42:0.000000', '43:0.000000', '44:0.000000', '45:0.000000', '46:0.007042', '#docid', '=', 'GX008-86-4444840', 'inc', '=', '1', 'prob', '=', '0.086622']


In [16]:
def parse(line):
    sl = line.split()
    y = int(sl[0])
    id = int(sl[1].split(':')[1])
    X = []
    for i in range(2, 48):
        val = float(sl[i].split(':')[1])
        X.append(val)
    return id, X, y

In [48]:
id, X, y = parse(data[0])
print(id)
print(X)
print(y)
print(len(data))

10002
[0.007477, 0.0, 1.0, 0.0, 0.00747, 0.0, 0.0, 0.0, 0.0, 0.0, 0.471076, 0.0, 1.0, 0.0, 0.477541, 0.00512, 0.0, 0.571429, 0.0, 0.004806, 0.768561, 0.727734, 0.716277, 0.582061, 0.0, 0.0, 0.0, 0.0, 0.780495, 0.962382, 0.999274, 0.961524, 0.0, 0.0, 0.0, 0.0, 0.797056, 0.697327, 0.721953, 0.582568, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007042]
0
9630


In [30]:
def populate(data):
    relv = {}
    not_relv = {}
    relv_index = {}
    not_relv_index = {}
    index = 0
    for line in data:
        id, X, y = parse(line)
        if y == 0:
            if id in not_relv:
                not_relv[id].append(X)
                not_relv_index[id].append(index)
            else:
                not_relv[id] = [X]
                not_relv_index[id] = [index]
        else:
            if id in relv:
                relv[id].append(X)
                relv_index[id].append(index)
            else:
                relv[id] = [X]
                relv_index[id] = [index]
        index += 1
    return relv, not_relv, relv_index, not_relv_index

In [34]:
relv, not_relv, relv_index, not_relv_index = populate(data)

In [40]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [47]:
relv_ids = relv.keys()
not_relv_ids = not_relv.keys()
both_ids = intersection(relv_ids, not_relv_ids)
est = 0
for id in both_ids:
    est += (len(relv_index[id]) * len(not_relv_index[id]))
print(est)

48086


In [70]:
def l2norm(X1, X2):
    l2 = 0.00
    for i in range(len(X1)):
        l2 += ((X1[i] - X2[i])**2)
    return math.sqrt(l2)

In [72]:
print(l2norm([1, 1], [0, 0]))

1.4142135623730951


In [78]:
def most_similar_antidocs(relv, not_relv, relv_index, not_relv_index):
    relv_ids = relv.keys()
    not_relv_ids = not_relv.keys()
    both_ids = intersection(relv_ids, not_relv_ids)
    
    similarity = []
    
    for id in both_ids:
        relv_docs = relv[id]
        relv_docs_index = relv_index[id]
        not_relv_docs = not_relv[id]
        not_relv_docs_index = not_relv_index[id]
        
        for i in range(len(relv_docs)):
            for j in range(len(not_relv_docs)):
                l2distance = l2norm(relv_docs[i], not_relv_docs[j])
                similarity.append((relv_docs_index[i], not_relv_docs_index[j], l2distance))
    
    similarity = sorted(similarity, key=lambda x: x[2])
    return similarity

In [90]:
similarity = most_similar_antidocs(relv, not_relv, relv_index, not_relv_index)

In [96]:
print(similarity[:10])

[(357, 355, 0.0), (1225, 1236, 0.010101000198000195), (7946, 7947, 0.012665432325822923), (1226, 1236, 0.014362060611207568), (4361, 4365, 0.020870549944838553), (4361, 4364, 0.027902033241324946), (5372, 5347, 0.04609267598002967), (2359, 2364, 0.06667934650249657), (7945, 7947, 0.07145043232199508), (2467, 2466, 0.07175823207131012)]


In [91]:
print(data[357])
print(data[355])

1 qid:10215 1:0.003221 2:0.000000 3:0.500000 4:1.000000 5:0.008039 6:0.000000 7:0.000000 8:0.000000 9:0.000000 10:0.000000 11:0.003457 12:0.000000 13:0.500000 14:1.000000 15:0.008001 16:0.000415 17:0.000000 18:0.615385 19:0.500000 20:0.000543 21:0.847059 22:0.986161 23:0.504325 24:0.896206 25:0.000000 26:0.000000 27:0.000000 28:0.000000 29:0.669733 30:0.820917 31:0.747268 32:0.895892 33:0.759851 34:0.285643 35:0.332812 36:0.234224 37:0.924614 38:0.969873 39:0.599569 40:0.955341 41:0.428571 42:0.459459 43:0.000000 44:0.000000 45:0.000000 46:0.000000 #docid = GX034-46-12610504 inc = 0.0216422811225112 prob = 0.105398

0 qid:10215 1:0.003221 2:0.000000 3:0.500000 4:1.000000 5:0.008039 6:0.000000 7:0.000000 8:0.000000 9:0.000000 10:0.000000 11:0.003457 12:0.000000 13:0.500000 14:1.000000 15:0.008001 16:0.000415 17:0.000000 18:0.615385 19:0.500000 20:0.000543 21:0.847059 22:0.986161 23:0.504325 24:0.896206 25:0.000000 26:0.000000 27:0.000000 28:0.000000 29:0.669733 30:0.820917 31:0.747268 3

In [92]:
def choosing_flips(similarity, num):
    flipped_indexes = []
    flipped_num = 0
    id = 0
    while flipped_num < num:
        not_relv_ind = similarity[id][1]
        if not_relv_ind not in flipped_indexes:
            flipped_indexes.append(not_relv_ind)
            flipped_num += 1
        id += 1
    return flipped_indexes

In [110]:
flipped = choosing_flips(similarity, int(len(data)*0))
flipped.sort()
print(flipped)

[]


In [111]:
def flip(data, flipped):
    for f in flipped:
        actual_label = int(data[f][0])
        if actual_label == 0:
            flipped_char = '1'
        else:
            flipped_char = '0'
        data[f] = flipped_char + data[f][1:] 

In [112]:
print(data[177])

0 qid:10078 1:0.162500 2:0.000000 3:0.000000 4:0.000000 5:0.162500 6:0.000000 7:0.000000 8:0.000000 9:0.000000 10:0.000000 11:0.163564 12:0.000000 13:0.000000 14:0.000000 15:0.163399 16:0.056837 17:0.000000 18:0.000000 19:0.380952 20:0.056762 21:0.581070 22:0.656250 23:0.694322 24:0.694835 25:0.000000 26:0.000000 27:0.000000 28:0.000000 29:0.000000 30:0.000000 31:0.000000 32:0.000000 33:0.000000 34:0.000000 35:0.000000 36:0.000000 37:0.584774 38:0.636051 39:0.697604 40:0.679162 41:0.500000 42:0.333333 43:0.000000 44:0.000000 45:0.000000 46:0.000000 #docid = GX269-47-0755600 inc = 0.0195516609085524 prob = 0.0897966



In [113]:
flip(data, flipped)
print(data[177])

0 qid:10078 1:0.162500 2:0.000000 3:0.000000 4:0.000000 5:0.162500 6:0.000000 7:0.000000 8:0.000000 9:0.000000 10:0.000000 11:0.163564 12:0.000000 13:0.000000 14:0.000000 15:0.163399 16:0.056837 17:0.000000 18:0.000000 19:0.380952 20:0.056762 21:0.581070 22:0.656250 23:0.694322 24:0.694835 25:0.000000 26:0.000000 27:0.000000 28:0.000000 29:0.000000 30:0.000000 31:0.000000 32:0.000000 33:0.000000 34:0.000000 35:0.000000 36:0.000000 37:0.584774 38:0.636051 39:0.697604 40:0.679162 41:0.500000 42:0.333333 43:0.000000 44:0.000000 45:0.000000 46:0.000000 #docid = GX269-47-0755600 inc = 0.0195516609085524 prob = 0.0897966

